In [3]:
import os
import matplotlib.pyplot as plt
import torchvision
import torch
import pandas as pd
import numpy as np

from utils.notebook import display_scrollable_dataframe,plot_sailency
from data_loaders import CUB_extnded_dataset
from models import get_inception_transform
from IPython.display import display

from sailency import get_saliency_maps,saliency_score_part

In [4]:
# Settings for the experiment your running
data_set = 'val' 

id = 1 # The id of the image you want to explain

# Settings for loading the dataset, make sure its the same as the one used to train the model. 
model_folder_1 = r"models\Sequential_Basemodel1"
model_folder_2 = r"models\Sequential_Basemodel2"

base_model_1 = int(model_folder_1[-1])
print(f'Base model: {base_model_1}')
base_model_2 = int(model_folder_2[-1])
print(f'Base model: {base_model_2}')

use_majority_voting_1 = True
use_majority_voting_2 = False

# You can find the settings in the .hydra folders config.yaml 
data_config_1 = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_1,
                'min_class_count':10,
                'return_visibility':True}


# You can find the settings in the .hydra folders config.yaml 
data_config_2 = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_2,
                'min_class_count':10,
                'return_visibility':True}

Base model: 1
Base model: 2


In [5]:
#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode='val',methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_1 = CUB_extnded_dataset(mode=data_set,config_dict=data_config_1,transform=transformer)
data_2 = CUB_extnded_dataset(mode=data_set,config_dict=data_config_2,transform=transformer)

#Get a dataset that return original images instead normalized ones
data_human_1 = CUB_extnded_dataset(mode=data_set,config_dict=data_config_1,transform=human_tansform)
concept_names_1 = data_1.consept_labels_names
class_names_1 = data_1.class_labels_names
n_classes_1 = data_1.n_classes
n_concepts_1 = data_1.n_concepts

data_human_2 = CUB_extnded_dataset(mode=data_set,config_dict=data_config_2,transform=human_tansform)
concept_names_2 = data_2.consept_labels_names
class_names_2 = data_2.class_labels_names
n_classes_2 = data_2.n_classes
n_concepts_2 = data_2.n_concepts


print(n_concepts_1,n_concepts_2)

159 312


In [ ]:
print(data_1[0][0].shape) # image
print(data_1[0][1].shape) # concept labels
print(data_1[0][2].shape) # class Y_one_hot
print(len(data_1[0][3])) # coordinates of the concepts

torch.Size([3, 299, 299])
torch.Size([159])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.])
None
torch.Size([200])
159
